# exp09
下記のNotebookにLag特徴量を追加
* https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977
* https://www.kaggle.com/code/takeshikobayashi/lag-features-are-all-you-need

In [15]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import joblib
import random
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from itertools import combinations

import ipywidgets as widgets
import snappy

import matplotlib.pyplot as plt
import seaborn as sns

from pytorch_tabnet.pretraining import TabNetPretrainer

from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from pytorch_tabnet.metrics import Metric
# from ipywidgets import interact, Select


In [2]:
# aa = pd.read_csv('../output/exp09_1800feature_lgb/oof_09_lgb_0.7470325254077378_5fold_seed46.csv')
# aa['tabnet_oot'].hist()

# 元特徴量とLag特徴量を結合
（最初だけ）

In [17]:
# ## 元データを結合するときのみ使用

# train = pd.read_parquet("../feature/exp03_amex-fe/train_fe_plus_plus.parquet")
# test = pd.read_parquet("../feature/exp03_amex-fe/test_fe_plus_plus.parquet")

# train_lag = pd.read_parquet("../feature/exp01_lag_feature/train_lag_feature.parquet")
# test_lag = pd.read_parquet("../feature/exp01_lag_feature/test_lag_feature.parquet")

# train = train.merge(train_lag,on = "customer_ID",how = "left")
# test = test.merge(test_lag,on = "customer_ID",how = "left")

In [4]:

train = pd.read_parquet("../feature/exp09_1800feature/train_1800.parquet")
test = pd.read_parquet("../feature/exp09_1800feature/test_1800.parquet")

In [11]:
# train_feature = train.columns.to_list()

# num_features0 = [col for col in train_feature if "last_lag_sub" in col]

# Training & Inference

In [10]:

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../input/amex-fe/'
    output_dir = '../output/exp09_1800feature_lgb/'
    seed = 46
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    model = "lgb"
    ver = "09"

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

# ====================================================
# XGB train
# ====================================================

def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

def cat_train(x, y, xt, yt,cat_features):
    print("# of features:", x.shape[1])
    assert x.shape[1] == xt.shape[1]
    
    prams = {
        'depth': 8,
        'iterations':5000,
#         'learning_rate': 0.05,
        'random_state':CFG.seed,
    }

#     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    watchlist = [(x, 'train'), (xt, 'eval')]
    clf = CatBoostClassifier(**prams)# 5000
    clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True, verbose_eval = 100)
#     print('best ntree_limit:', clf.best_ntree_limit)
#     print('best score:', clf.best_score)
    # return clf.predict_proba(xt)[:, 1]
    return clf.predict_proba(xt)[:, 1], clf

def tabnet_train(x, y, xt, yt,cat_features):
    print("# of features:", x.shape[1])
    assert x.shape[1] == xt.shape[1]
    
    prams = {
        'depth': 8,
        'iterations':5000,
#         'learning_rate': 0.05,
        'random_state':CFG.seed,
    }

#     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    watchlist = [(x, 'train'), (xt, 'eval')]
    clf = CatBoostClassifier(**prams)# 5000
    clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True, verbose_eval = 100)
#     print('best ntree_limit:', clf.best_ntree_limit)
#     print('best score:', clf.best_score)
    # return clf.predict_proba(xt)[:, 1]
    return clf.predict_proba(xt)[:, 1], clf



# using amex metric to evaluate tabnet
class Amex_tabnet(Metric):
    
    def __init__(self):
        self._name = 'amex_tabnet'
        self._maximize = True

    def __call__(self, y_true, y_pred):
        amex = amex_metric_numpy(y_true, y_pred[:, 1])
        return max(amex, 0.)

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


def xgb_amex(y_pred, y_true):
    return 'amex', amex_metric_np(y_pred,y_true.get_label())


# Created by https://www.kaggle.com/yunchonggan
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def amex_metric_np(preds: np.ndarray, target: np.ndarray) -> float:
    indices = np.argsort(preds)[::-1]
    preds, target = preds[indices], target[indices]

    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_mask = cum_norm_weight <= 0.04
    d = np.sum(target[four_pct_mask]) / np.sum(target)

    weighted_target = target * weight
    lorentz = (weighted_target / weighted_target.sum()).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    n_pos = np.sum(target)
    n_neg = target.shape[0] - n_pos
    gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)

    g = gini / gini_max
    return 0.5 * (g + d)

def amex_metric_numpy(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

In [11]:
seed_everything(CFG.seed)
# train, test = read_data()


In [12]:

# cat_features = [
#     "B_30",
#     "B_38",
#     "D_114",
#     "D_116",
#     "D_117",
#     "D_120",
#     "D_126",
#     "D_63",
#     "D_64",
#     "D_66",
#     "D_68"
# ]

# cat_features = [f"{cf}_last" for cf in cat_features]
# for cat_col in cat_features:
#     encoder = LabelEncoder()
#     train[cat_col] = encoder.fit_transform(train[cat_col])
#     test[cat_col] = encoder.transform(test[cat_col])
# # Round last float features to 2 decimal place
# num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
# num_cols = [col for col in num_cols if 'last' in col]
# for col in num_cols:
#     train[col + '_round2'] = train[col].round(2)
#     test[col + '_round2'] = test[col].round(2)
# # Get the difference between last and mean
# num_cols = [col for col in train.columns if 'last' in col]
# num_cols = [col[:-5] for col in num_cols if 'round' not in col]
# for col in num_cols:
#     try:
#         train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
#         test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
#     except:
#         pass
# # Transform float64 and float32 to float16
# num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
# for col in tqdm(num_cols):
#     train[col] = train[col].astype(np.float16)
#     test[col] = test[col].astype(np.float16)


# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
params = {
    'objective': 'binary',
    'metric': CFG.metric,
    'boosting': CFG.boosting_type,
    'seed': CFG.seed,
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
    }


# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
# all_data = features.extend("customer_ID")


train.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)

# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))

preds = []
cid = []
cids = []

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
    
    cid = train["customer_ID"].loc[val_ind]
    cids.extend(cid)
#     lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
#     lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
    
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_val, y_val)
    
    model = lgb.train(
        params = params,
        train_set = lgb_train,
        num_boost_round = 10500,#10500
        valid_sets = [lgb_train, lgb_valid],
        early_stopping_rounds = 1500,
        verbose_eval = 500,
        feval = lgb_amex_metric
        )
    
    joblib.dump(model, f'{CFG.output_dir}{CFG.model}_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
    
    val_pred = model.predict(x_val)
    oof_predictions[val_ind] = val_pred
    
    test_pred = model.predict(test[features])
    test_predictions += test_pred / CFG.n_folds
    
    
    score = amex_metric(y_val, val_pred)
    print(f'Our fold {fold} CV score is {score}')
    del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
    gc.collect()
    

# Compute out of folds metric
score = amex_metric(train[CFG.target], oof_predictions)
print(f'Our out of folds CV score is {score}')
# test_predictions = np.mean(preds,axis = 0)

dic_oof = {
    "customer_ID":cids,
    "target":train[CFG.target],
    "tabnet_oot":oof_predictions
}

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame(dic_oof)
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
test_df.to_csv(f'{CFG.output_dir}test_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)



NameError: name 'train' is not defined

In [ ]:
# oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


# # Create a dataframe to store test prediction
# test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'{CFG.output_dir}test_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)

